# T2.4 Extraer con selenium datos de licitacións do Concello da Coruña
Queremos facer un código que garde nunha BBDD a Microsoft SQL Server a información dos expedientes de licitación do Concello da Coruña e nun directorio (no actual) as capturas de pantalla de cada expediente.

Para montar o Microsoft SQL Server sigue a guía de: https://jfsanchez.es/docs/docker-sqlserver-microsoft/

Debes empregar selenium e a páxina: https://contrataciondelestado.es

Debes navegar por ela coma se indica no documento.

Por cada elemento (expediente), meterse nel e facer captura de pantalla (automáticamente, coa API de selenium).

Entrega:

    O código nun jupyter notebook.
    A BBDD exportada de Microsoft SQL Server.
    Un zip coas capturas de pantalla que fixo selenium.


In [32]:
from webdriver_manager.firefox import GeckoDriverManager
GeckoDriverManager().install()

'C:\\Users\\daniel.martinezcarre\\.wdm\\drivers\\geckodriver\\win64\\v0.34.0\\geckodriver.exe'

## Importacións

In [33]:
import argparse
from enum import Enum
import sys
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random
from bs4 import BeautifulSoup
import pprint
from io import StringIO
from bs4 import BeautifulSoup
import pandas as pd


### Variables e funcións

In [34]:
URL="https://contrataciondelestado.es/"

In [35]:
def waitFinishLoad(driver,idLoader,needsDisapeared=False,by=By.ID,needsWait=False):
    #Espera a la carga de un elemento antes de continuar
    if idLoader is None:
        return
    WebDriverWait(driver, timeout=10).until(EC.presence_of_element_located((by, idLoader)))
    if needsDisapeared:
        WebDriverWait(driver, timeout=10).until(EC.invisibility_of_element((by, idLoader)))
    if needsWait:
        time.sleep(random.choice([0.1,0.2,0.3,0.4,0.5,0.6]))
    else:
        time.sleep(random.choice([0.05,0.1,0.2]))

def elementBy(query,driver,**kargs):
    if not kargs.get("needsWait",False):
        waitFinishLoad(driver,kargs.get("idLoader",None),by=kargs.get("by",By.ID),needsWait=kargs.get("needsWait",False))
    try:
        element=driver.find_element(kargs.get("byq",By.XPATH),query)
        if kargs.get("screenshot"):
            driver.execute_script("arguments[0].scrollIntoView();", element)
    except:
        return None
    waitFinishLoad(driver,kargs.get("idLoader",None),by=kargs.get("by",By.ID),needsWait=kargs.get("needsWait",False))
    return element



def goToNextPage(driver):
    element=elementBy("//input[@value='Siguiente']",driver,idLoader="//table[@id='myTablaBusquedaCustom']",by=By.XPATH)
    driver.execute_script("arguments[0].scrollIntoView();", element)
    element.click()


In [36]:
def esperar_carga_pagina(driver, url):
    """
    Espera a que la página web se cargue completamente.

    Args:
        driver: El objeto del navegador web.
        url: La URL de la página web que se quiere cargar.
    """
    # Carga la página web
    driver.get(url)

    # Espera a que la página se cargue completamente
    wait = WebDriverWait(driver, 10)
    wait.until(lambda driver: driver.execute_script("return document.readyState") == "complete")


def automatizaciones(driver):
    """
    Ejecuta las automatizaciones en la página web con esperas.

    Args:
        driver: El objeto del navegador web.
    """

    # Click en 'Publicaciones'
    wait = WebDriverWait(driver, 10)
    link = wait.until(EC.visibility_of_element_located((By.XPATH, "//a[@href='/wps/portal/licitaciones']")))
    link.click()

    # Click en 'Licitaciones'
    wait = WebDriverWait(driver, 10)
    link_licitaciones = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "divLogo")))
    link_licitaciones.click()

    # Click en 'Búsqueda avanzada'
    wait = WebDriverWait(driver, 10)
    link_busqueda_avanzada = wait.until(EC.presence_of_element_located((By.ID, "viewns_Z7_AVEQAI930OBRD02JPMTPG21004_:form1:textBusquedaAvanzada")))
    link_busqueda_avanzada.click()

    # Click en 'Seleccionar'
    wait = WebDriverWait(driver, 10)
    link_busqueda_avanzada = wait.until(EC.presence_of_element_located((By.ID, "viewns_Z7_AVEQAI930OBRD02JPMTPG21004_:form1:idSeleccionarOCLink")))
    link_busqueda_avanzada.click()

    # Click en 'Entidades'
    wait = WebDriverWait(driver, 10)
    entidades = wait.until(EC.presence_of_element_located((By.XPATH, "//div[text()='ENTIDADES LOCALES']/../../td[@class='multiline']")))
    entidades.click()

    # Click en 'Galicia'
    wait = WebDriverWait(driver, 10)
    galicia = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//div[text()='Galicia']/../../td[@class='multiline']")))
    galicia[1].click()

    # Click en 'A Coruña'
    wait = WebDriverWait(driver, 10)
    link_coruna_1 = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//div[text()='A Coruña']/../../td[@class='multiline']")))
    link_coruna_1[2].click()

    # Click en 'Ayuntamientos'
    wait = WebDriverWait(driver, 10)
    link_junta = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//div[text()='Ayuntamientos']/../../td[@class='multiline']")))
    link_junta[3].click()

    # Click en 'A Coruña' por 2ª vez
    wait = WebDriverWait(driver, 10)
    link_junta = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//div[text()='A Coruña']")))
    link_junta[1].click()

    # Click en 'Junta de Gobierno del Ayuntamiento de A Coruña'
    wait = WebDriverWait(driver, 10)
    link_junta = wait.until(EC.presence_of_element_located((By.XPATH, "//option[text()='Junta de Gobierno del Ayuntamiento de A Coruña']")))
    link_junta.click()

    # Click en 'Añadir'
    wait = WebDriverWait(driver, 10)
    boton_anhadir = wait.until(EC.presence_of_element_located((By.XPATH, "//input[@name='viewns_Z7_AVEQAI930OBRD02JPMTPG21004_:form1:botonAnadirMostrarPopUpArbolEO']")))
    boton_anhadir.click()

    # Click en 'Buscar'
    wait = WebDriverWait(driver, 10)
    boton_buscar = wait.until(EC.presence_of_element_located((By.XPATH, "//input[@title='Buscar']")))
    boton_buscar.click()


## Carga de la URL y automatizaciones

In [39]:
driver = webdriver.Firefox()
esperar_carga_pagina(driver, URL)
automatizaciones(driver)

## Parseado de tabla e introducción de valores con BeautifulSoup4

In [40]:
body = driver.execute_script("return document.body")
page = body.get_attribute("innerHTML")

#### Parseo a HTML

In [41]:
soup = BeautifulSoup(page, "html.parser")

### Extracción de cabeceiras

In [42]:
def sacar_cabeceras():
    """
    Imprime las cabeceras de la tabla
    """
    header = []
    job_elements = soup.find('table', id="myTablaBusquedaCustom")

    for i, job_element in enumerate(job_elements.find_all('tr')):
        if i != 0: break
        else:
            header = [el.text.strip() for el in job_element.find_all('th')]
    print(header)
sacar_cabeceras()

['Expediente', 'Tipo de Contrato', 'Estado', 'Importe', 'Fechas', 'Órgano de Contratación']


### Rascado de datos 

In [43]:
def crearInforme():
    """
    Retorna los datos de la tabla de una página
    """
    rows = []
    body = driver.execute_script("return document.body")
    source = body.get_attribute('innerHTML')
    soup = BeautifulSoup(source, 'html.parser')
    job_elements = soup.find('table', id="myTablaBusquedaCustom")
    for i, job_element in enumerate(job_elements.find_all('tr')):
        if i>1:
            row = []        
            for el in job_element.find_all('td'):
                row.append(el.text.strip())
            rows.append(row)
    return rows

rows = crearInforme()

## Capturas de expedientes y volcado de datos en variable

In [44]:
datos = []
lista = []

def cap_exp():
    """
    Este método realiza varias funciones:
        - Ejecuta, en una pestaña aparte, todos los expedientes.
        - Hace capturas de todos los expedientes y los guarda en el directorio específicado.
        - Se mueve de página a medida que va iterando todos los elementos de la actual.
        - Retorna un listado con todos los datos de todas las páginas."""
    try:
        datos = []
        enlaces = []
        ind = 0
        path = "C:\\Users\\daniel.martinezcarre\\Desktop\\Repositorios\\sbd_repositorio\\SBD\\practicas\\T2.4\\capturas\\"

        while True:

            # Inicializo filas para actualizar nuevos datos
            filas = []

            # Mi actual driver
            base_handle = driver.current_window_handle

            td = driver.find_elements(By.XPATH,'.//td[@class="tdExpediente"]')
            for i in td:
                enlace = i.find_element(By.XPATH,'.//div//a[2]').get_attribute("href")
                enlaces.append(enlace)

            wait = WebDriverWait(driver, 10)

            for link in enlaces:
                ind += 1
                try:
                    # Abrir enlace en pestaña nueva
                    driver.execute_script(f"window.open('{link}', 'new_window')")

                    # Cambiar el foco a la nueva pestaña
                    driver.switch_to.window(driver.window_handles[-1])

                    # Esperar a que la página se cargue
                    WebDriverWait(driver, 30).until(
                    EC.presence_of_element_located((By.ID, "footer-newShow")))
                    
                    # Capturar la pantalla
                    driver.save_screenshot(path+f"{ind}_expediente.png")

                    driver.close()

                    # Cambiar el foco a la pestaña principal
                    driver.switch_to.window(driver.window_handles[0])
                except Exception as e:
                    print(f"Error al procesar enlace: {link} - {e}")
            
            goToNextPage(driver)
            filas = crearInforme()

            # Descarga de los datos recogidos al conjunto de datos
            for i in filas:
                datos.append(i)
            return datos
    except:
        print("No hay más páginas")


# Cargo todos los datos en la variable para insertarlos en la BBDD
for i in range(45):
    lista = cap_exp()
    for l in lista:
        datos.append(l)    
    

No hay más páginas


TypeError: 'NoneType' object is not iterable

In [45]:
len(datos)

867

## SQL Server
> docker run -e "ACCEPT_EULA=Y" -e "MSSQL_SA_PASSWORD=Abcd1234." -e "MSSQL_PID=Evaluation" -p 41433:1433 --name sqlpreview --hostname sqlpreview -d mcr.microsoft.com/mssql/server:2022-preview-ubuntu-22.04

A contrasinal debe ter unha maiúscula e 8 caracteres, debemos instalar **pyodbc** para manejar el SQL Server.<br>***conda install -c conda-forge -y pyodbc***

### Instalación de driver necesario

https://learn.microsoft.com/en-us/sql/connect/odbc/download-odbc-driver-for-sql-server?view=sql-server-ver16#download-for-windows

In [46]:
import pyodbc

In [47]:
DB_HOST = '10.133.29.229,41433'
DB_NAME = 'datos_corunha'
DB_USER = 'sa'
DB_PASSWORD = 'Abcd1234.'

### Establecemos conexión

In [48]:
connectionString = f'''DRIVER={{ODBC Driver 18 for SQL Server}};
SERVER={DB_HOST};DATABASE={DB_NAME};
UID={DB_USER};PWD={DB_PASSWORD};
Encrypt=Yes;TrustServerCertificate=Yes'''

print(connectionString)
conexion = pyodbc.connect(connectionString)


DRIVER={ODBC Driver 18 for SQL Server};
SERVER=10.133.29.229,41433;DATABASE=datos_corunha;
UID=sa;PWD=Abcd1234.;
Encrypt=Yes;TrustServerCertificate=Yes


### Creación do dataframe de datos

In [49]:
titulos = ['expediente', 'tipo_contrato', 'estado', 'importe', 'fechas', 'organo_contratacion']

df = pd.DataFrame(datos, columns= titulos)

> Modificación do importe para poder importar od datos a BBDD

In [50]:
# Reemplazo de caracteres necesarios para o posterior cambio de tipo de dato
df['importe'] = df['importe'].str.replace('.','').str.replace(',','.')

# Cambio de tipo de dato a float
df['importe']= df['importe'].astype('float')

## Insertando datos na base de datos

In [51]:
#Creamos un cursos para logo acceder aos resultados
cursor = conexion.cursor()

SQL_PREPARED_STATEMENT = """
INSERT INTO contratos(expediente, tipo_contrato, estado, importe, fechas, organo)
VALUES (?,?,?,?,?,?)
"""

# Hacerlo con executemany()
cursor.executemany("""
                   INSERT INTO contratos(expediente,tipo_contrato,estado,importe,fechas,organo)
                   Values (?,?,?,?,?,?)
""",df.values.tolist())
cursor.commit()

#Pechar o cursor e a conexión
cursor.close()
conexion.close()